In [83]:
import pandas as pd
import scipy.signal as sng
import numpy as np

In [84]:
eeg=pd.read_csv('извлеченные_ээг/eeg_S01.csv')

In [85]:
#electrodes=np.delete(eeg.columns, [0, 1, -1, -2, -3])
electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
fs=300 #частота дискретизации ээг

In [86]:
#Почему-то данные в виде строки, так что нужно перегнать обратно в числа#
def THE_MOST_STUPID_FUNCTION(eeg_producta):
    list_eeg={}
    for i in electrodes:
        eeg_producta_i=np.delete(np.array(eeg_producta[i].split('[')), [0, 1])
        s=[]
        for j in range(len(eeg_producta_i)):
            vrem=eeg_producta_i[j].split(',')[:-1]
            vrem[-1]=vrem[-1][:-1]
            for k in range(len(vrem)):
                vrem[k]=float(vrem[k])
            s.append(vrem)
        list_eeg[f'{i}']=s
    return list_eeg

In [87]:
#Выделим частоты и спектральные плотности мощности (psd)#
def get_welt(eeg_producta, fs=300):
    product_elecrod_psd={}
    #electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
    electrodes=['F3', 'F4'] #Что за электроды AF3 и AF4??????????????#
    for i in electrodes:
        eeg_i=eeg_producta[i]
        a=[]
        b=[]
        g=[]
        t=[]
        for j in eeg_i:
            welt=sng.welch(j, fs=fs)
            freq=welt[0]
            psd=welt[1]

            alpha_freq=freq[freq>8]
            alpha_freq=alpha_freq[alpha_freq<13]
            if len(alpha_freq)!=0:
                start=np.where(freq==alpha_freq[0])
                stop=np.where(freq==alpha_freq[-1])
                alpha_psd=psd[start[0][0]:stop[0][0]+1]
                a.append({'freq_alpha_diapazon': alpha_freq, 'psd': alpha_psd})
            
            beta_freq=freq[freq>13]
            beta_freq=beta_freq[beta_freq<22]
            if len(beta_freq)!=0:
                start=np.where(freq==beta_freq[0])
                stop=np.where(freq==beta_freq[-1])
                beta_psd=psd[start[0][0]:stop[0][0]+1]
                b.append({'freq_beta_diapazon': beta_freq, 'psd': beta_psd}) #ДОБАВИТЬ КУДА_НИБудь#

            gamma_freq=freq[freq>22]
            gamma_freq=gamma_freq[gamma_freq<30]
            if len(gamma_freq)!=0:
                start=np.where(freq==gamma_freq[0])
                stop=np.where(freq==gamma_freq[-1])
                gamma_psd=psd[start[0][0]:stop[0][0]+1]
                g.append({'freq_gamma_diapazon': gamma_freq, 'psd': gamma_psd}) #ДОБАВИТЬ КУДА_НИБудь#

            theta_freq=freq[freq>4]
            theta_freq=theta_freq[theta_freq<8]
            if len(theta_freq)!=0:
                start=np.where(freq==theta_freq[0])
                stop=np.where(freq==theta_freq[-1])
                theta_psd=psd[start[0][0]:stop[0][0]+1]
                t.append({'freq_theta_diapazon': theta_freq, 'psd': theta_psd}) #ДОБАВИТЬ КУДА_НИБудь#

        product_elecrod_psd[f'{i}']=[a, b, g, t]
    return product_elecrod_psd

In [88]:
#То же, но для всех продуктов#
def get_all_psd(eeg):
    s={}
    for i in range(len(eeg)):
        eeg_producta=eeg.loc[i]
        krivie_ruki=THE_MOST_STUPID_FUNCTION(eeg_producta=eeg_producta)
        pp=eeg['product_page'].loc[i]
        pn=eeg['product_number'].loc[i]
        
        eeg_psded=get_welt(krivie_ruki)
        s[f'{pp} {pn}']=[eeg['index'].loc[i], eeg['info'].loc[i], eeg_psded]
    return s

In [ ]:
evrika=get_all_psd(eeg=eeg)

In [90]:
evrika['Page1 0'][2]['F3'][3]

[{'freq_theta_diapazon': array([4.6875  , 5.859375, 7.03125 ]),
  'psd': array([4.89327469, 1.75098359, 1.53925382])},
 {'freq_theta_diapazon': array([4.54545455]), 'psd': array([1.84628761])},
 {'freq_theta_diapazon': array([4.87804878, 7.31707317]),
  'psd': array([0.75336399, 1.48854081])}]

In [91]:
#Усредним все psd для каждого продукта как в каждом временном отрезке наблюдения, так и по всем ним#

#Я НЕ ВЫКИДЫВАЛ ВРЕМЕННЫЕ ОТРЕЗКИ НАБЛЮДЕНИЯ КОНКРЕТНОГО ПРОДУКТА, КОТОРЫЕ ЯВНО КОРОЧЕ ОСТАЛЬНЫХ#

def averager(eeg_pesded):
    diapazons=['alpha', 'beta', 'gamma', 'theta']
    for i in eeg_pesded:
        eeg_page_prod=eeg_pesded[i][2]
        electrodes=['F3', 'F4'] 
        for k in electrodes:
            for indx, diap in enumerate(diapazons):
                vrem=[]
                for j in range(len(eeg_page_prod[k][indx])):
                    #if len(eeg_page_prod[k][j]['psd'])>2:
                    #    evrika[i][2][k][j][f'mean_psd_{k}']=np.mean(eeg_page_prod[k][j]['psd'])
                    #    vrem.append(np.mean(eeg_page_prod[k][j]['psd']))
                    evrika[i][2][k][indx][j][f'mean_psd_{k}']=np.mean(eeg_page_prod[k][indx][j]['psd'])
                    vrem.append(np.mean(eeg_page_prod[k][indx][j]['psd']))
                evrika[i][2][f'{k}_{diap}_average_all_of_saw']=round(np.mean(vrem), ndigits=5)
averager(evrika)

c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [93]:
#Посчитаем фронтальную альфа-ассиметрию (aw) для каждого продукта#
def get_AW(eeg_avereged):
    s=[]
    inf=[]
    for i in eeg_avereged:
        mean_page_prod=eeg_avereged[i][2]
        aw_prod=float((mean_page_prod['F4_alpha_average_all_of_saw']-mean_page_prod['F3_alpha_average_all_of_saw'])/(mean_page_prod['F4_alpha_average_all_of_saw']+mean_page_prod['F3_alpha_average_all_of_saw']))
        s.append(round(aw_prod, ndigits=5))
        inf.append(int(eeg_avereged[i][1][11]))
    return [s, inf]

In [94]:
def get_Ram12_valence(eeg_avereged):
    s=[]
    inf=[]
    for i in eeg_avereged:
        mean_page_prod=eeg_avereged[i][2]
        r12_val_prod=float(mean_page_prod['F4_alpha_average_all_of_saw']-mean_page_prod['F3_beta_average_all_of_saw'])
        s.append(round(r12_val_prod, ndigits=5))
        inf.append(int(eeg_avereged[i][1][11]))
    return [s, inf]

In [95]:
def get_Ram15_valence(eeg_avereged):
    s=[]
    inf=[]
    for i in eeg_avereged:
        mean_page_prod=eeg_avereged[i][2]
        r15_val_prod=float((mean_page_prod['F4_alpha_average_all_of_saw']/mean_page_prod['F4_beta_average_all_of_saw'])-(mean_page_prod['F3_alpha_average_all_of_saw']/mean_page_prod['F3_beta_average_all_of_saw']))
        s.append(round(r15_val_prod, ndigits=5))
        inf.append(int(eeg_avereged[i][1][11]))
    return [s, inf]

In [96]:
def get_effort_index1(eeg_avereged):
    s=[]
    inf=[]
    for i in eeg_avereged:
        mean_page_prod=eeg_avereged[i][2]
        ef_prod=float((mean_page_prod['F4_theta_average_all_of_saw']-mean_page_prod['F3_theta_average_all_of_saw'])/(mean_page_prod['F4_theta_average_all_of_saw']+mean_page_prod['F3_theta_average_all_of_saw']))
        s.append(round(ef_prod, ndigits=5))
        inf.append(int(eeg_avereged[i][1][11]))
    return [s, inf]

In [97]:
hell_aw=get_AW(evrika)
hell_val12=get_Ram12_valence(evrika)
hell_val15=get_Ram15_valence(evrika)
hell_effort=get_effort_index1(evrika)

In [98]:
df_aw=pd.DataFrame({'product_page_number': list(evrika)})
df_aw['aw']=hell_aw[0]
df_aw['buy']=hell_aw[1]

In [99]:
df_val12=pd.DataFrame({'product_page_number': list(evrika)})
df_val12['val12']=hell_val12[0]
df_val12['buy']=hell_val12[1]

df_val15=pd.DataFrame({'product_page_number': list(evrika)})
df_val15['val15']=hell_val15[0]
df_val15['buy']=hell_val15[1]

df_effort=pd.DataFrame({'product_page_number': list(evrika)})
df_effort['effort']=hell_effort[0]
df_effort['buy']=hell_effort[1]

In [100]:
df=df_aw.merge(df_val12)
df=df.merge(df_val15)
df=df.merge(df_effort)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 126 entries, 0 to 131
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_page_number  126 non-null    object 
 1   aw                   126 non-null    float64
 2   buy                  126 non-null    int64  
 3   val12                126 non-null    float64
 4   val15                126 non-null    float64
 5   effort               126 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 6.9+ KB


In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [102]:
x=df.drop(columns=['product_page_number', 'buy', 'val12', 'val15', 'effort'])
y=df.drop(columns=['product_page_number', 'aw', 'val12', 'val15', 'effort'])

In [103]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=10)

In [104]:
#Посмотрим, может быть и получилось#
LR=LogisticRegression()
LR.fit(x_train, y_train)
y_predict_LR=LR.predict(x_test)
print(metrics.accuracy_score(y_test, y_predict_LR))

0.9230769230769231


c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Сделаем для всех испытуемых#
people=['01', '02', '03', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
fs=300 #частота дискретизации ээг
df=pd.DataFrame()

for t in people:
        eeg=pd.read_csv(f'извлеченные_ээг/eeg_S{t}.csv')
        evrika=get_all_psd(eeg=eeg)
        averager(evrika)

        hell_aw=get_AW(evrika)
        hell_val12=get_Ram12_valence(evrika)
        hell_val15=get_Ram15_valence(evrika)
        hell_effort=get_effort_index1(evrika)
        df_val12=pd.DataFrame({'product_page_number': list(evrika)})
        df_val12['val12']=hell_val12[0]
        df_val12['buy']=hell_val12[1]
        df_val15=pd.DataFrame({'product_page_number': list(evrika)})
        df_val15['val15']=hell_val15[0]
        df_val15['buy']=hell_val15[1]
        df_effort=pd.DataFrame({'product_page_number': list(evrika)})
        df_effort['effort']=hell_effort[0]
        df_effort['buy']=hell_effort[1]

        df_people=pd.DataFrame({'people': np.full(len(list(evrika)), t), 'product_page_number': list(evrika)})
        df_people['aw']=hell_aw[0]
        df_people['buy']=hell_aw[1]
        df_people=df_people.merge(df_val12)
        df_people=df_people.merge(df_val15)
        df_people=df_people.merge(df_effort)
        
        df=pd.concat([df, df_people])

In [57]:
#def aw_God(directory='извлеченные_ээг'):
#    people=['01', '02', '03', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
#    electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
#    fs=300 #частота дискретизации ээг
#    df=pd.DataFrame()
#    for t in people:
#        eeg=pd.read_csv(f'{directory}/eeg_S{t}.csv')
#        evrika=get_all_psd(eeg=eeg)
#        averager(evrika)
#        hell=get_AW(evrika)
#        df_people=pd.DataFrame({'people': np.full(len(list(evrika)), t), 'product_page_number': list(evrika)})
#        df_people['aw']=hell[0]
#        df_people['buy']=hell[1]
#        df=pd.concat(df, df_people)
#    return df

In [106]:
df

,people,product_page_number,aw,buy,val12,val15,effort
0,01,Page1 0,-0.21720,0,0.05437,-0.81024,0.14033
1,01,Page1 1,0.30423,0,0.18803,1.10880,0.05269
2,01,Page1 2,0.15855,0,0.06618,-0.14021,0.52483
3,01,Page1 3,-0.13846,1,0.89158,-0.89813,0.22057
4,01,Page1 4,0.06312,0,0.47358,-0.08872,0.01328
...,...,...,...,...,...,...,...
137,44,Page6 19,-0.12071,1,-0.08974,0.30855,-0.32381
138,44,Page6 20,-0.10263,0,-0.45575,-0.04902,-0.20760
139,44,Page6 21,0.11665,0,-0.15555,0.25523,0.21105
140,44,Page6 22,0.27023,0,0.70788,-0.05353,0.52069


In [107]:
#df.to_csv('eeg_aw_try3.csv')

In [111]:
df=pd.read_csv('eeg_aw_valence_effort.csv')
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5606 entries, 0 to 5901
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           5606 non-null   int64  
 1   people               5606 non-null   int64  
 2   product_page_number  5606 non-null   object 
 3   aw                   5606 non-null   float64
 4   buy                  5606 non-null   int64  
 5   val12                5606 non-null   float64
 6   val15                5606 non-null   float64
 7   effort               5606 non-null   float64
dtypes: float64(4), int64(3), object(1)
memory usage: 394.2+ KB


In [121]:
#Посмотрим, может быть и получилось#
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
x=df.drop(columns=['product_page_number', 'buy', 'Unnamed: 0', 'people', 'aw', 'val12', 'val15'])
y=df.drop(columns=['product_page_number', 'Unnamed: 0', 'people', 'aw', 'val12', 'val15', 'effort'])

In [122]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=10)

In [123]:
LR=LogisticRegression()
LR.fit(x_train, y_train)
y_predict_LR=LR.predict(x_test)
print(metrics.accuracy_score(y_test, y_predict_LR))

0.8556149732620321


c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
